In [2]:
import xarray as xarr
import glob
import os
from fnmatch import fnmatch
import rioxarray as rio
import numpy as np

In [132]:
parent_dir = '../Data/Training/VIIRS/Rasterized/NC/'

def list_files(parent_dir, pattern = ".nc"):
    pattern = pattern
    filelist = []

    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            #append the file name to the list
            filelist.append(os.path.join(root,file))

    for r in [f for f in filelist if f[-len(pattern):] != pattern]:
        filelist.remove(r)
        
    return filelist

files = list_files(parent_dir)

In [123]:
for f in files:
    frp_bool = f.split('_')[-1][:-3]
    
    
    raster = rio.open_rasterio(f)
    
        
    parent_dir = ('/').join(list(map(lambda x: x.replace('NC', 'COG'), 
                                     f.split('/')[:7]))) + \
                                            '/' + frp_bool + '/'
    
    if not os.path.isdir(parent_dir):
        os.makedirs(parent_dir)
        
    new_file_path = parent_dir + f.split('/')[-1][:-3] + '.tif'
    
    raster.rio.to_raster(new_file_path, driver = "COG")
    


In [179]:
# TESTING TO MAKE SURE EVERYTHING WROTE PROPERLY
COG_files = list_files('../Data/Training/VIIRS/Rasterized/COG/', pattern = '.tif')
nc_files = list_files('../Data/Training/VIIRS/Rasterized/NC/', pattern = '.nc')

nc_files.sort()
COG_files.sort()

cog_new = []
nc_new = []
for f, ff in zip(nc_files, COG_files):
    f = f[:-3]
    ff = ff[:-4]
    
    nc_new.append(f)
    cog_new.append(ff)

cog_new = [f.replace('COG/', '') for f in cog_new]
nc_new = [f.replace('NC/', '') for f in nc_new]

print([f for f in cog_new if f not in nc_new])
print([f for f in nc_new if f not in cog_new])

print(len(COG_files))
print(len(nc_files))

[]
[]
4256
4256


# Upload to s3 bucket